In [45]:
query = """
{
  propositions(first: 100000) {
    totalCount
    edges {
      node {
        # title
        # status
        governance: governance {
          platform_deployment_id: id
          # platform_deployment_name: name
        }
        phases {
          __typename
          ...on GovernorProposal {
          	proposal_id: id
            # state
            start {
              date_as_epoch: timestamp
            }
            proposer {
              author: id
            }
            voteTally {
              # __typename
              votes_count: ballotCount
            }
        	}
        }
        # proposal_id: id
      }
    }
  }
}

# x platform: governor
# x platform_deployment_id: governance id
# x proposal_id
# x author: from GovernorProposal
# x date -- this should be as a pandas date, not as an epoch
# x votes_count: ballotCount from GovernorProposal phase
"""

In [46]:
# hit gql https://graphql.messari.io/query
import requests

response = requests.post('https://graphql.messari.io/query', json={'query': query})
print(response.status_code)
data = response.json()
len(data['data']['propositions']['edges'])

200


9053

In [47]:
import pandas as pd
# df from edges
df = pd.DataFrame(data['data']['propositions']['edges'])
# expand the node column
df = pd.concat([df.drop(['node'], axis=1), df['node'].apply(pd.Series)], axis=1)
df.head()

,governance,phases
0,{'platform_deployment_id': 'e229944a-420a-4945...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
1,{'platform_deployment_id': '612f3113-2625-4175...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."
3,{'platform_deployment_id': '6d02d511-788b-41c5...,[{'__typename': 'SnapshotProposal'}]
4,{'platform_deployment_id': 'a0542f99-69f1-474f...,"[{'__typename': 'PreliminaryDiscussion'}, {'__..."


In [48]:
# explode phases column
phases_df = df.explode('phases')
phases_df.head()

,governance,phases
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'}
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'}
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'}


In [49]:
# rename phases to phase
phases_df.rename(columns={'phases': 'phase'}, inplace=True)
phases_df.phase.value_counts()

{'__typename': 'SnapshotProposal'}                                                                                                                                                                                                                      7069
{'__typename': 'PreliminaryDiscussion'}                                                                                                                                                                                                                 6827
{'__typename': 'MakerPoll'}                                                                                                                                                                                                                              115
{'__typename': 'MakerProposal'}                                                                                                                                                                                                                  

In [50]:
phases_df.head()

,governance,phase
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'}
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'}
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'}
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'}


In [51]:
# drop where phase is nan
phases_df.dropna(subset=['phase'], inplace=True)

In [52]:
# pull out typename into new col
phases_df['typename'] = phases_df['phase'].apply(lambda x: x.get('__typename'))
phases_df.head()

,governance,phase,typename
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion
0,{'platform_deployment_id': 'e229944a-420a-4945...,{'__typename': 'SnapshotProposal'},SnapshotProposal
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion
1,{'platform_deployment_id': '612f3113-2625-4175...,{'__typename': 'SnapshotProposal'},SnapshotProposal
2,{'platform_deployment_id': '9f6f4845-12b9-4f8c...,{'__typename': 'PreliminaryDiscussion'},PreliminaryDiscussion


In [53]:
# keep only where typename is 'GovernorProposal'
governor_proposals = phases_df[phases_df['typename'] == 'GovernorProposal']
governor_proposals.head()

,governance,phase,typename
20,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
33,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
45,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
66,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal
96,{'platform_deployment_id': '32b7bb94-fbc9-4172...,"{'__typename': 'GovernorProposal', 'proposal_i...",GovernorProposal


In [54]:
governor_proposals.shape

(740, 3)

In [55]:
# x platform: governor
# x platform_deployment_id: governing_body_id
# x proposal_id
# x author: from GovernorProposal
# x date -- this should be as a pandas date, not as an epoch
# x votes_count: ballotCount from GovernorProposal phase

In [56]:
# expand phase dict column into separate columns
governor_proposals = pd.concat([governor_proposals.drop(['phase'], axis=1), governor_proposals['phase'].apply(pd.Series)], axis=1)
governor_proposals.head()

,governance,typename,__typename,proposal_id,start,proposer,voteTally
20,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,GovernorProposal,GovernorProposal,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124}
33,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,GovernorProposal,GovernorProposal,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98}
45,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,GovernorProposal,GovernorProposal,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9}
66,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,GovernorProposal,GovernorProposal,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413}
96,{'platform_deployment_id': '32b7bb94-fbc9-4172...,GovernorProposal,GovernorProposal,1062472272376004331395813518426159754568225993...,{'date_as_epoch': 1677010487},{'author': '0x17768cfd6030e2b65eb5086e34a512fd...,{'votes_count': 3}


In [57]:
# drop typename and __typename cols
governor_proposals = governor_proposals.drop(columns=['typename', '__typename'])
governor_proposals.head()

,governance,proposal_id,start,proposer,voteTally
20,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124}
33,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98}
45,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9}
66,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413}
96,{'platform_deployment_id': '32b7bb94-fbc9-4172...,1062472272376004331395813518426159754568225993...,{'date_as_epoch': 1677010487},{'author': '0x17768cfd6030e2b65eb5086e34a512fd...,{'votes_count': 3}


In [58]:
# * platform
# * platform_deployment_id
# * proposal_id
# * author
# * date -- this should be as a pandas date, not as an epoch
# * votes_count

In [59]:
governor_proposals['platform_deployment_id'] = governor_proposals['governance'].map(lambda x: x['platform_deployment_id'])
governor_proposals['date_as_epoch'] = governor_proposals['start'].map(lambda x: x['date_as_epoch'])
# map author out of proposer.author
governor_proposals['author'] = governor_proposals['proposer'].map(lambda x: x['author'])
# map votes_count out of voteTally
governor_proposals['votes_count'] = governor_proposals['voteTally'].map(lambda x: x['votes_count'])
governor_proposals.head()

,governance,proposal_id,start,proposer,voteTally,platform_deployment_id,date_as_epoch,author,votes_count
20,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-110,{'date_as_epoch': 1666203011},{'author': '0x55b16934c3661e1990939bc57322554d...,{'votes_count': 124},e1a6f432-bc3c-438e-a07b-ce35095de79a,1666203011,0x55b16934c3661e1990939bc57322554d9b09f262,124
33,{'platform_deployment_id': 'c0b43294-f8b9-4bbb...,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,{'date_as_epoch': 1678300151},{'author': '0x4be88f63f919324210ea3a2ccad4ff07...,{'votes_count': 98},c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,1678300151,0x4be88f63f919324210ea3a2ccad4ff0734425f91,98
45,{'platform_deployment_id': '1a11c997-d9c2-4d3c...,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,{'date_as_epoch': 1663340879},{'author': '0xe8ea8bae250028a8709a3841e0ae1a44...,{'votes_count': 9},1a11c997-d9c2-4d3c-abe4-3d85190d02f2,1663340879,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,9
66,{'platform_deployment_id': 'e1a6f432-bc3c-438e...,0xec568fffba86c094cf06b22134b23074dfe2252c-147,{'date_as_epoch': 1674470891},{'author': '0xf71fc92e2949ccf6a5fd369a0b402ba8...,{'votes_count': 413},e1a6f432-bc3c-438e-a07b-ce35095de79a,1674470891,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,413
96,{'platform_deployment_id': '32b7bb94-fbc9-4172...,1062472272376004331395813518426159754568225993...,{'date_as_epoch': 1677010487},{'author': '0x17768cfd6030e2b65eb5086e34a512fd...,{'votes_count': 3},32b7bb94-fbc9-4172-9219-0ed422edf0ce,1677010487,0x17768cfd6030e2b65eb5086e34a512fde5dc1f1f,3


In [60]:
governor_proposals = governor_proposals[['platform_deployment_id', 'proposal_id', 'author', 'date_as_epoch', 'votes_count']]
governor_proposals.head()

,platform_deployment_id,proposal_id,author,date_as_epoch,votes_count
20,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,1666203011,124
33,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,1678300151,98
45,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,1663340879,9
66,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,1674470891,413
96,32b7bb94-fbc9-4172-9219-0ed422edf0ce,1062472272376004331395813518426159754568225993...,0x17768cfd6030e2b65eb5086e34a512fde5dc1f1f,1677010487,3


In [61]:
governor_proposals['platform'] = 'governor'

In [62]:
# parse date from seconds epoch
governor_proposals['date'] = pd.to_datetime(governor_proposals['date_as_epoch'], unit='s')
governor_proposals.head()

,platform_deployment_id,proposal_id,author,date_as_epoch,votes_count,platform,date
20,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,1666203011,124,governor,2022-10-19 18:10:11
33,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,1678300151,98,governor,2023-03-08 18:29:11
45,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,1663340879,9,governor,2022-09-16 15:07:59
66,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,1674470891,413,governor,2023-01-23 10:48:11
96,32b7bb94-fbc9-4172-9219-0ed422edf0ce,1062472272376004331395813518426159754568225993...,0x17768cfd6030e2b65eb5086e34a512fde5dc1f1f,1677010487,3,governor,2023-02-21 20:14:47


In [63]:
# * platform
# * platform_deployment_id
# * proposal_id
# * author
# * date -- this should be as a pandas date, not as an epoch
# * votes_count

# keep these cols
governor_proposals = governor_proposals[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count']]
governor_proposals.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
20,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x55b16934c3661e1990939bc57322554d9b09f262,2022-10-19 18:10:11,124
33,governor,c0b43294-f8b9-4bbb-8a25-b5773d7b23b1,0xdbd27635a534a3d3169ef0498beb56fb9c937489-49,0x4be88f63f919324210ea3a2ccad4ff0734425f91,2023-03-08 18:29:11,98
45,governor,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,0x3d5fc645320be0a085a32885f078f7121e5e5375-26,0xe8ea8bae250028a8709a3841e0ae1a44820d677b,2022-09-16 15:07:59,9
66,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-147,0xf71fc92e2949ccf6a5fd369a0b402ba80bc61e02,2023-01-23 10:48:11,413
96,governor,32b7bb94-fbc9-4172-9219-0ed422edf0ce,1062472272376004331395813518426159754568225993...,0x17768cfd6030e2b65eb5086e34a512fde5dc1f1f,2023-02-21 20:14:47,3


In [64]:
# group by platform_deployment_id and val counts
governor_proposals.groupby('platform_deployment_id')['proposal_id'].count()

platform_deployment_id
1a11c997-d9c2-4d3c-abe4-3d85190d02f2     37
32b7bb94-fbc9-4172-9219-0ed422edf0ce      9
33c90aeb-e7eb-4656-8663-f4831dc63468     13
4e758c1e-8410-4fa4-9924-6bead3afde57     36
4fa685ba-26e9-40cb-b98c-cb9182a3b407     31
535d7de4-0c53-42ab-8e9a-001430a17163      7
56ba6663-b056-435c-846e-09d2ab1cf434     15
5b4e7a42-0f44-4e1b-9297-d006997b2233      4
9611750e-035c-48d1-ac3e-27ba85c1dec9    166
9ed2dea1-f955-4453-b99d-a8bac116bbe4     20
9fc93056-60a4-4972-894f-a3db0733fb37     14
a53c23d6-d21f-408a-906c-1272985c2506     15
ab030329-6200-45a0-aca5-f22aa89581fa     75
b5f50e58-820f-413f-8ba6-84c44c59c1b3      5
bddd5124-ab7f-4911-b961-d923c1dd6085     18
c0b43294-f8b9-4bbb-8a25-b5773d7b23b1     58
c4b7bc0b-887f-4bc6-a165-d4b726e6f35e     17
d897ffd3-8c0d-4a6a-92e0-0c84e0729e61     13
e1a6f432-bc3c-438e-a07b-ce35095de79a    173
e99546c7-58d9-4f78-b3b2-084dfad11436      5
eb93f72a-f8fb-49ff-9a23-4b5f78a4aa04      9
Name: proposal_id, dtype: int64

In [65]:
# group by platform_deployment_id and val counts
governor_proposals.groupby('platform_deployment_id')['votes_count'].sum()

platform_deployment_id
1a11c997-d9c2-4d3c-abe4-3d85190d02f2       220
32b7bb94-fbc9-4172-9219-0ed422edf0ce        42
33c90aeb-e7eb-4656-8663-f4831dc63468        64
4e758c1e-8410-4fa4-9924-6bead3afde57       296
4fa685ba-26e9-40cb-b98c-cb9182a3b407     27100
535d7de4-0c53-42ab-8e9a-001430a17163        81
56ba6663-b056-435c-846e-09d2ab1cf434      1877
5b4e7a42-0f44-4e1b-9297-d006997b2233        59
9611750e-035c-48d1-ac3e-27ba85c1dec9     11328
9ed2dea1-f955-4453-b99d-a8bac116bbe4      3484
9fc93056-60a4-4972-894f-a3db0733fb37    349445
a53c23d6-d21f-408a-906c-1272985c2506       274
ab030329-6200-45a0-aca5-f22aa89581fa      1677
b5f50e58-820f-413f-8ba6-84c44c59c1b3        33
bddd5124-ab7f-4911-b961-d923c1dd6085       480
c0b43294-f8b9-4bbb-8a25-b5773d7b23b1      4541
c4b7bc0b-887f-4bc6-a165-d4b726e6f35e        49
d897ffd3-8c0d-4a6a-92e0-0c84e0729e61       148
e1a6f432-bc3c-438e-a07b-ce35095de79a     13611
e99546c7-58d9-4f78-b3b2-084dfad11436        18
eb93f72a-f8fb-49ff-9a23-4b5f78a4aa04 

In [66]:
# save to csv
governor_proposals.to_csv('governor_proposals.csv', index=False)